In [7]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os, shutil
import cv2
import glob as gb
import pandas as pd
import seaborn as sns
import matplotlib.image as mpimg
from PIL import Image

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import warnings
warnings.filterwarnings("ignore")

In [17]:
# vreate folder data with zipped images form kaggle!!!

path_file_cat = 'data/PetImages/Cat'
path_file_dog = 'data/PetImages/Dog'

base_dir = 'data/PetImages/Train'

# Train folder
Train_dir = os.path.join(base_dir, 'Train')

cat_train_dir = os.path.join(Train_dir, 'cat')
dog_train_dir = os.path.join(Train_dir, 'dog')

# Test Folder
Test_dir = os.path.join(base_dir, 'Test')

cat_test_dir = os.path.join(Test_dir, 'cat')
dog_test_dir = os.path.join(Test_dir, 'dog')

# Validation Folder
valid_dir = os.path.join(base_dir, 'validation')
cat_val_dir = os.path.join(valid_dir, 'cat')
dog_val_dir = os.path.join(valid_dir, 'dog')

In [18]:
# Creating folders
os.mkdir(base_dir)

# Train folder
os.mkdir(Train_dir)
os.mkdir(cat_train_dir)
os.mkdir(dog_train_dir)

# Test Folder
os.mkdir(Test_dir)
os.mkdir(cat_test_dir)
os.mkdir(dog_test_dir)

# validation folder
os.mkdir(valid_dir)
os.mkdir(cat_val_dir)
os.mkdir(dog_val_dir)

In [19]:
train_list_cat_file = [f'{i}.jpg' for i in range(6500)]
train_list_dog_file = [f'{i}.jpg' for i in range(6500)]

test_list_cat_file = [f'{i}.jpg' for i in range(6500, 9500)]
test_list_dog_file = [f'{i}.jpg' for i in range(6500, 9500)]

val_list_cat_file = [f'{i}.jpg' for i in range(9500, 12500)]
val_list_dog_file = [f'{i}.jpg' for i in range(9500, 12500)]

In [20]:
# organaize cat and dog in train folders
# cat
for filename in train_list_cat_file:
    src = os.path.join(path_file_cat, filename)
    dst = os.path.join(cat_train_dir, filename)
    shutil.copyfile(src, dst)
    
# dog
for filename in train_list_dog_file:
    src = os.path.join(path_file_dog, filename)
    dst = os.path.join(dog_train_dir, filename)
    shutil.copyfile(src, dst)
    
# organaize cat and dog in test folders
# cat
for filename in test_list_cat_file:
    src = os.path.join(path_file_cat, filename)
    dst = os.path.join(cat_test_dir, filename)
    shutil.copyfile(src, dst)
    
# dog
for filename in test_list_dog_file:
    src = os.path.join(path_file_dog, filename)
    dst = os.path.join(dog_test_dir, filename)
    shutil.copyfile(src, dst)
    
# organaize cat and dog in validation folders
# cat
for filename in val_list_cat_file:
    src = os.path.join(path_file_cat, filename)
    dst = os.path.join(cat_val_dir, filename)
    shutil.copyfile(src, dst)
    
# dog
for filename in val_list_dog_file:
    src = os.path.join(path_file_dog, filename)
    dst = os.path.join(dog_val_dir, filename)
    shutil.copyfile(src, dst)

In [21]:
print("number of cat img in train folder: ", len(os.listdir(cat_train_dir)))
print("number of dog img in train folder: ", len(os.listdir(dog_train_dir)))
print("number of cat img in test folder: ", len(os.listdir(cat_test_dir)))
print("number of dog img in test folder: ", len(os.listdir(dog_test_dir)))
print("number of cat img in validation folder: ", len(os.listdir(cat_val_dir)))
print("number of dog img in validation folder: ", len(os.listdir(dog_val_dir)))

number of cat img in train folder:  6500
number of dog img in train folder:  6500
number of cat img in test folder:  3000
number of dog img in test folder:  3000
number of cat img in validation folder:  3000
number of dog img in validation folder:  3000
